In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Extract recurrent mobility matrix

First, the raw spreadsheet `data/raw/census_2011/Pop_LPW_NL_25FEB15.xlsx` was modified in MS Excel. The format of the data is as follows:

- rows: municipality of residence
- columns: municipality of work

The dataset contained, for each Belgian province, a column of 'unknowns', indicating we know where these individuals live but not where they work. These 10 columns were removed manually. Further, the column `Werkt in Belgie` was renamed `Belgie` to make name-based row and column matching easier. The resulting dataset was placed in `data/raw/census_2011/Pop_LPW_NL_25FEB15_remove_unknowns.xlsx`. The recurrent mobility matrix extracted from the resulting spreadsheet thus has two assumptions,

- People working abroad are ignored. These 92.000 individuals account for 2% of the working population.
- People whos workplace is unknown are ignored. An alternative assumptions could be the following: the unknowns work in their place of residence.

In [ ]:
abs_dir = os.getcwd()
rel_dir = os.path.join(abs_dir, '../data/interim/census_2011/Pop_LPW_NL_25FEB15_delete_unknown.xlsx')
df = pd.read_excel(rel_dir, sheet_name="Tabel1_2011")

In [ ]:
codes=df['00.24 - Werkende bevolking volgens geslacht, verblijfplaats en plaats van tewerkstelling'].loc[5:1942].dropna().values
codes_int = [int(i) for i in codes]
mobility_df=pd.DataFrame(np.zeros([len(codes),len(codes)]),index=codes,columns=codes)
names = df.iloc[5:,1].dropna().values
rows=[]
for i in df['00.24 - Werkende bevolking volgens geslacht, verblijfplaats en plaats van tewerkstelling'].loc[5:1942].dropna().index:
    rows.append(df.iloc[i+2,4:-1].values)

matrix = np.zeros([len(rows),len(rows)])
for j in range(len(rows)):
    matrix[j,:]=rows[j]

In [ ]:
mobility_df=pd.DataFrame(matrix,index=codes_int,columns=codes_int)
mobility_df.head()

In [ ]:
idx_arrondisement=[]
for idx in mobility_df.index:
    if ((str(idx)[-3:] == '000') & (len(str(idx)) != 4) & (str(idx)[-4:] != '0000') & (str(idx)[0] != '0')):
        idx_arrondisement.append(idx)
print(idx_arrondisement)

In [ ]:
mobility_df.loc[idx_arrondisement,idx_arrondisement]

In [ ]:
mobility_df.loc[idx_arrondisement,idx_arrondisement].to_csv('../data/interim/census_2011/recurrent_mobility.csv', index=True)